# Creating Monthly Composites
### ECOSTRESS Tutorials
###### This code is to be used when you have a collection of images that you would like to make MONTHLY composites of. It works best if you have already cloud masked or QCed your images (See the cloud masking or QC tutorials). This code may be modified to create yearly composites, seasonal composites, etc. 
###### This code is written to create monthly composites of Land Surface Temperature (LST) files, but may be modified for use with other ECOSTRESS data products.

### Import Necessary Libraries

In [ ]:
import os
from os import makedirs
import numpy as np
import pandas as pd
import rioxarray
import rioxarray.merge
from glob import glob
from os.path import join, basename, splitext
from datetime import datetime

### Define Input and Output Locations, and List Files

In [ ]:
#Define input and output folders
input_directory = r"Replace_this_text_with_folder_path" 
output_directory = r"Replace_this_text_with_folder_path" 
#Ensure the output directory exists
makedirs(output_directory, exist_ok=True)

#Set aesthetics 
alpha = 0.7 # Sets the transparency of the image to 70%
fig_width_px = 1080 # Defines the size of the figure
fig_height_px = 720

#Find all masked ST files in the input directory
ST_masked_filenames = sorted(glob(join(input_directory, "*_ST.tif")))

### Extract the Month and Year from Filenames and Group Them

In [ ]:
#Creates an empty dictionary to store groups of files
file_groups = {}
#Iterates through each file name
for filename in ST_masked_filenames:
    # Extract the datetime string from the filename
    datetime_str = splitext(basename(filename))[0].split("_")[-2]
    #Transforms the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, "%Y.%m.%d.%H.%M.%S")
        #Make sure the date format matches your input images. 
        #If you used our batch cloud mask or QC code, this date format should be correct.
    #Format the datetime object to get a string of the month and year
    month_year = datetime_obj.strftime("%Y-%m")

    #If the month and year value is not already in the dictionary ...
    if month_year not in file_groups:
        # ... add it as an empty list
        file_groups[month_year] = []
    #Add the current file name to the list for this month and year
    file_groups[month_year].append(filename)

### Create Median Composites for Every Month Group

In [ ]:
#For each month group, loop through every file
for month_year, files in file_groups.items():
    #Open the rasters using rioxarray
    ST_masked_rasters = [rioxarray.open_rasterio(filename).squeeze("band", drop=True) for filename in files]
    #Merge all the rasters into one array
    ST_composite = rioxarray.merge.merge_arrays(ST_masked_rasters, nodata=np.nan)
    #Calculate the median for each pixel
    ST_composite.data = np.nanmedian(np.stack([raster.rio.reproject_match(ST_composite).data for raster in ST_masked_rasters]), axis=0)
    #Replace 0s with NaN
    ST_composite.data = np.where(ST_composite.data == 0, np.nan, ST_composite.data)

    #Create an output file path by combining the output folder location with a file name
    output_file_path = join(output_directory, f"File_name_{month_year}_Median.tif") #Can rename if needed!
    #Save the Composite Raster
    ST_composite.rio.to_raster(output_file_path)
    print(f"Saved composite for {month_year}")
    